<a href="https://colab.research.google.com/github/asegura4488/MetodosComputacionales1/blob/main/Semana15/MCMC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install emcee
!pip install corner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 2.0 MB/s eta 0:00:00


In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import emcee
import corner
import os
from google.colab import drive
import scipy.optimize as spo

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [8]:
os.chdir("/content/drive/MyDrive/ColabNotebooks/Cursos/MetodosComputacionalesI")
!ls

Parcial2    Semana10  Semana13	Semana2  Semana5  Semana8
ProyectoML  Semana11  Semana14	Semana3  Semana6  Semana9
Semana1     Semana12  Semana15	Semana4  Semana7


In [9]:
file = '/content/drive/MyDrive/ColabNotebooks/Cursos/MetodosComputacionalesI/Semana15/Data/Gaussiano.csv'
data = pd.read_csv(file)
x = data['x']

In [10]:
X = np.array(data['x'])

In [11]:
def Gaus(p,x):
  mu, sigma = p
  return 1/(sigma*np.sqrt(2*np.pi))*np.exp(-(x-mu)**2/(2*sigma**2))

In [12]:
def LogPrior(p):
  mu, sigma = p
  if -10 < mu < 10 and 0 < sigma < 20:
    return 0.
  else:
    return -np.inf

In [13]:
def Likelihood(p,x):
  return Gaus(p,x)

In [14]:
def JointLikelihood(p,x):
  return +np.sum(np.log(Likelihood(p,x))) # La distribucion posterior es positiva

In [17]:
JointLikelihood([1,1.5],X)

np.float64(-406.9096896360665)

In [18]:
def LogPosterior(p,x):
  LogP = LogPrior(p)
  if not np.isfinite(LogP):
    return -np.inf
  else:
    return LogP + JointLikelihood(p,x)

In [21]:
LogPosterior([100,3],X)

-inf